In [47]:
"""Analyze cookie preference menu."""

import pandas as pd

from consent.util.default_path import get_data_dir
from ooutil.df_util import read_jsons_in_dir


data_dir = get_data_dir('2021-03-18') / 'pref_menu_scan'
assert data_dir.exists()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [48]:
prefs = read_jsons_in_dir(data_dir)
prefs.head()

Read 2997 files.


,domain,pref_menu_elem,failed,pref_menu_type,category,cookie_check,failed_reason
0,home.blog,None,True,NaN,NaN,NaN,NaN
1,python.ca,None,True,NaN,NaN,NaN,NaN
2,elitedaily.com,None,True,NaN,NaN,NaN,NaN
3,iata.org,None,True,NaN,NaN,NaN,NaN
4,jagranjosh.com,None,True,NaN,NaN,NaN,NaN


In [49]:
category_data = []
for _, row in prefs.dropna(subset=['category']).iterrows():
    for category in row['category']:
        category['domain'] = row['domain']
        category_data.append(category)
categories = pd.DataFrame(category_data)

In [50]:
print("Num successfully extracted: ", categories.domain.nunique())

Num successfully extracted:  184


In [51]:
prefs[~prefs.category.isna()].reset_index().head()

,index,domain,pref_menu_elem,failed,pref_menu_type,category,cookie_check,failed_reason
0,9,hindawi.com,onetrust,NaN,tab,"[{'id': 'C0001', 'name': 'Strictly Necessary Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'hindawi.com'}, {'id': 'C0002', 'name': 'Performance Cookies', 'cur_status': 'true', 'next_status': 'false', 'domain': 'hindawi.com'}, {'id': 'C0004', 'name': 'Targeting Cookies', 'cur_status': 'true', 'next_status': 'false', 'domain': 'hindawi.com'}]",True,NaN
1,18,nflxvideo.net,onetrust,NaN,tab,"[{'id': 'C0001', 'name': 'Essential Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'nflxvideo.net'}, {'id': 'C0002', 'name': 'Performance and Functionality Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'nflxvideo.net'}, {'id': 'C0004', 'name': 'Advertising Cookies', 'cur_status': 'true', 'next_status': 'false', 'domain': 'nflxvideo.net'}]",True,NaN
2,26,bitnami.com,onetrust,NaN,tab,"[{'id': 'C0001', 'name': 'Strictly Necessary', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'bitnami.com'}, {'id': 'C0002', 'name': 'Performance', 'cur_status': 'true', 'next_status': 'false', 'domain': 'bitnami.com'}, {'id': 'C0004', 'name': 'Advertising', 'cur_status': 'true', 'next_status': 'false', 'domain': 'bitnami.com'}, {'id': 'C0005', 'name': 'Social Media', 'cur_status': 'true', 'next_status': 'false', 'domain': 'bitnami.com'}]",True,NaN
3,35,mailchi.mp,onetrust,NaN,tab,"[{'id': '1', 'name': 'Essential Website Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'mailchi.mp'}, {'id': '2', 'name': 'Performance and Functionality Cookies', 'cur_status': 'false', 'next_status': 'false', 'domain': 'mailchi.mp'}, {'id': '4', 'name': 'Advertising (Targeting) Cookies', 'cur_status': 'false', 'next_status': 'false', 'domain': 'mailchi.mp'}, {'id': '3', 'name': 'Analytics and Customization Cookies', 'cur_status': 'false', 'next_status': 'false', '...",True,NaN
4,39,illinois.edu,onetrust,NaN,accordion,"[{'id': 'C0001', 'name': 'Strictly Necessary Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'illinois.edu'}, {'id': 'C0002', 'name': 'Performance Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'illinois.edu'}, {'id': 'C0003', 'name': 'Functional Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domain': 'illinois.edu'}, {'id': 'C0004', 'name': 'Targeting Cookies', 'cur_status': 'always_active', 'next_status': 'true', 'domai...",True,NaN


In [52]:
categories.head()

,id,name,cur_status,next_status,domain
0,C0001,Strictly Necessary Cookies,always_active,true,hindawi.com
1,C0002,Performance Cookies,true,false,hindawi.com
2,C0004,Targeting Cookies,true,false,hindawi.com
3,C0001,Essential Cookies,always_active,true,nflxvideo.net
4,C0002,Performance and Functionality Cookies,always_active,true,nflxvideo.net


In [53]:
n_domains = categories.domain.nunique()
def all_choices_always_active(adf):
    return adf.cur_status.nunique() == 1 and adf.cur_status.unique()[0] == 'always_active'
all_actives = []

def get_all_choices_aa_domains():
    grouped = categories.groupby('domain')
    for name, group in grouped:
        if all_choices_always_active(group):
            yield name
all_choices_aa_domains = list(get_all_choices_aa_domains())
n_all_aa = len(all_choices_aa_domains)
print(f"Num domains that have all-choice-always-active: {n_all_aa} {n_all_aa / n_domains * 100:.2f}%")
all_choices_aa_domains[:3]

Num domains that have all-choice-always-active: 52 28.26%


['al.com', 'allure.com', 'architecturaldigest.com']

In [54]:
pref_vc = prefs.pref_menu_elem.value_counts()
pref_vc

onetrust           257
onetrust_legacy     23
cookiebot           11
quantcast            8
sourcepoint          6
trustarc             2
Name: pref_menu_elem, dtype: int64

In [55]:
n_detects = categories.domain.nunique()
n_sdk = pref_vc['onetrust'] + pref_vc['onetrust_legacy']
print(f"Num extracted over all detected onetrust: {n_detects}/{n_sdk} {n_detects/n_sdk*100:.2f}%", )

Num extracted over all detected onetrust: 184/280 65.71%


In [57]:
prefs[(prefs.category.isna()) & (prefs.pref_menu_elem == 'onetrust')].reset_index().head(50)

,index,domain,pref_menu_elem,failed,pref_menu_type,category,cookie_check,failed_reason
0,48,hypebeast.com,onetrust,True,NaN,NaN,NaN,NaN
1,55,nydailynews.com,onetrust,True,NaN,NaN,NaN,NaN
2,78,crunchyroll.com,onetrust,True,NaN,NaN,NaN,NaN
3,90,redbull.com,onetrust,True,NaN,NaN,NaN,NaN
4,95,cc.com,onetrust,True,NaN,NaN,NaN,NaN
5,118,azlyrics.com,onetrust,True,NaN,NaN,NaN,NaN
6,133,codecademy.com,onetrust,True,NaN,NaN,NaN,NaN
7,143,prnewswire.com,onetrust,True,NaN,NaN,NaN,NaN
8,174,sheknows.com,onetrust,True,NaN,NaN,NaN,NaN
9,186,247sports.com,onetrust,True,NaN,NaN,NaN,NaN
